In [1]:
import cv2
import mediapipe as mp

# Initialize MediaPipe hands model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=4, min_detection_confidence=0.7, min_tracking_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils

# Define function to check for a "fist" gesture with reduced sensitivity
def is_fist(hand_landmarks, threshold=0.05):
    for hand_landmark in hand_landmarks:
        # Extract landmarks for fingertips (index 8, middle 12, ring 16, pinky 20)
        index_tip = hand_landmark.landmark[8]
        middle_tip = hand_landmark.landmark[12]
        ring_tip = hand_landmark.landmark[16]
        pinky_tip = hand_landmark.landmark[20]

        # Extract landmarks for knuckles (index 6, middle 10, ring 14, pinky 18)
        index_knuckle = hand_landmark.landmark[6]
        middle_knuckle = hand_landmark.landmark[10]
        ring_knuckle = hand_landmark.landmark[14]
        pinky_knuckle = hand_landmark.landmark[18]

        # Check if all fingertips are below their corresponding knuckles with a threshold for reduced sensitivity
        if (index_tip.y > index_knuckle.y + threshold and
            middle_tip.y > middle_knuckle.y + threshold and
            ring_tip.y > ring_knuckle.y + threshold and
            pinky_tip.y > pinky_knuckle.y + threshold):
            return True
    return False

# Define function to check for a "cross" gesture with increased sensitivity
def is_cross_sign(hand_landmarks):
    if len(hand_landmarks) == 2:
        # Extract landmarks for both hands
        hand1_landmarks = hand_landmarks[0].landmark
        hand2_landmarks = hand_landmarks[1].landmark

        # Check if the x-coordinates of specific landmarks indicate crossing
        # These landmarks are selected to check crossing
        if (abs(hand1_landmarks[8].x - hand2_landmarks[8].x) < 0.1 and  # Compare index finger tips
            abs(hand1_landmarks[12].x - hand2_landmarks[12].x) < 0.1):  # Compare middle finger tips
            return True
    return False

# Start capturing video from webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    # Flip the image horizontally for a mirrored view
    image = cv2.flip(image, 1)
    # Convert the image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image and detect hands
    results = hands.process(image_rgb)

    # Draw the hand annotations on the image
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Check for specific gestures (fist and cross)
        if is_fist(results.multi_hand_landmarks):
            # Generate alert for fist gesture
            cv2.putText(image, "Alert: Person is asking for help", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        if is_cross_sign(results.multi_hand_landmarks):
            # Generate alert for cross sign gesture
            cv2.putText(image, "Alert: Person is asking for help", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the image
    cv2.imshow('Hand Gesture Detection', image)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


c:\Users\kachh\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
